## Baseline Model
- baseline xgboost model with some default params, no cross validation etc
- will do a train and validation split using the train_set and use that as validation set to do early stopping

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import datetime
import missingno as msno
import lightgbm as lgb
import xgboost as xgb
from sklearn import preprocessing
import gc
from sklearn.model_selection import KFold, TimeSeriesSplit, train_test_split
from sklearn.metrics import roc_auc_score
from time import time
from sklearn import metrics

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ieee-cis-fraudulent-transactions-eda-data-prep/train_full.pkl
/kaggle/input/ieee-cis-fraudulent-transactions-eda-data-prep/custom.css
/kaggle/input/ieee-cis-fraudulent-transactions-eda-data-prep/__results__.html
/kaggle/input/ieee-cis-fraudulent-transactions-eda-data-prep/test_full.pkl
/kaggle/input/ieee-cis-fraudulent-transactions-eda-data-prep/__output__.json
/kaggle/input/ieee-cis-fraudulent-transactions-eda-data-prep/__notebook__.ipynb
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv


In [3]:
train_full = pd.read_pickle('../input/ieee-cis-fraudulent-transactions-eda-data-prep/train_full.pkl')
test_full = pd.read_pickle('../input/ieee-cis-fraudulent-transactions-eda-data-prep/test_full.pkl')

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
########################### Model Features 
# Remove Some Features
rm_cols = [
    'TransactionID','TransactionDT', # These columns are pure noise right now
    'isFraud',                          # Not target in features))
    'DT_M'                           # Column that we used to simulate test set
]

# Final features
features_columns = [col for col in list(train_full.columns) if col not in rm_cols]

In [5]:
# Label Encoding
for f in test_full.columns:
    if train_full[f].dtype=='object' or test_full[f].dtype=='object': 
        train_full[f] = train_full[f].fillna('unseen_before_label')
        test_full[f]  = test_full[f].fillna('unseen_before_label')
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_full[f].values) + list(test_full[f].values))
        train_full[f] = lbl.transform(list(train_full[f].values))
        test_full[f] = lbl.transform(list(test_full[f].values)) 

In [9]:
# split train and validation, by time (ie. holdout the last month of data)
## We need Divide Train Set by Time blocks
## Convert TransactionDT to Months
## And use last month as Validation
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
train_full['DT_M'] = train_full['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
train_full['DT_M'] = (train_full['DT_M'].dt.year-2017)*12 + train_full['DT_M'].dt.month 

main_train_set = train_full[train_full['DT_M']<(train_full['DT_M'].max())].reset_index(drop=True)
validation_set = train_full[train_full['DT_M']==train_full['DT_M'].max()].reset_index(drop=True)

# X_train, X_test, y_train, y_test = train_test_split(train_full[features_columns], train_full['isFraud'], test_size=0.2, random_state=42)

del train_full
gc.collect()



[0]	validation_0-auc:0.808404	validation_1-auc:0.808093
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.844447	validation_1-auc:0.845715
[2]	validation_0-auc:0.850719	validation_1-auc:0.851079
[3]	validation_0-auc:0.852333	validation_1-auc:0.853269
[4]	validation_0-auc:0.85366	validation_1-auc:0.854343
[5]	validation_0-auc:0.85448	validation_1-auc:0.855556
[6]	validation_0-auc:0.855618	validation_1-auc:0.856499
[7]	validation_0-auc:0.856066	validation_1-auc:0.856714
[8]	validation_0-auc:0.856516	validation_1-auc:0.857153
[9]	validation_0-auc:0.857366	validation_1-auc:0.857781
[10]	validation_0-auc:0.857977	validation_1-auc:0.858635
[11]	validation_0-auc:0.859082	validation_1-auc:0.860216
[12]	validation_0-auc:0.85914	validation_1-auc:0.860332
[13]	validation_0-auc:0.86367	validation_1-auc:0.86519
[14]	validation_0-auc:0.864072	validation_1-auc:0.865516
[1

[141]	validation_0-auc:0.962686	validation_1-auc:0.943658
[142]	validation_0-auc:0.962894	validation_1-auc:0.943898
[143]	validation_0-auc:0.96295	validation_1-auc:0.943964
[144]	validation_0-auc:0.963124	validation_1-auc:0.944151
[145]	validation_0-auc:0.96329	validation_1-auc:0.944284
[146]	validation_0-auc:0.963467	validation_1-auc:0.944447
[147]	validation_0-auc:0.963668	validation_1-auc:0.944671
[148]	validation_0-auc:0.963786	validation_1-auc:0.944778
[149]	validation_0-auc:0.963882	validation_1-auc:0.944869
[150]	validation_0-auc:0.963991	validation_1-auc:0.944977
[151]	validation_0-auc:0.964168	validation_1-auc:0.945152
[152]	validation_0-auc:0.964264	validation_1-auc:0.945241
[153]	validation_0-auc:0.964402	validation_1-auc:0.945339
[154]	validation_0-auc:0.964581	validation_1-auc:0.945513
[155]	validation_0-auc:0.964966	validation_1-auc:0.945818
[156]	validation_0-auc:0.965087	validation_1-auc:0.945938
[157]	validation_0-auc:0.965245	validation_1-auc:0.946066
[158]	validation

[283]	validation_0-auc:0.977588	validation_1-auc:0.955693
[284]	validation_0-auc:0.97765	validation_1-auc:0.955772
[285]	validation_0-auc:0.977726	validation_1-auc:0.955857
[286]	validation_0-auc:0.977827	validation_1-auc:0.955953
[287]	validation_0-auc:0.977853	validation_1-auc:0.955986
[288]	validation_0-auc:0.977909	validation_1-auc:0.956038
[289]	validation_0-auc:0.977925	validation_1-auc:0.956053
[290]	validation_0-auc:0.977943	validation_1-auc:0.956072
[291]	validation_0-auc:0.977964	validation_1-auc:0.956066
[292]	validation_0-auc:0.978015	validation_1-auc:0.95613
[293]	validation_0-auc:0.978051	validation_1-auc:0.956184
[294]	validation_0-auc:0.978155	validation_1-auc:0.956281
[295]	validation_0-auc:0.9782	validation_1-auc:0.956306
[296]	validation_0-auc:0.978217	validation_1-auc:0.956338
[297]	validation_0-auc:0.978233	validation_1-auc:0.956363
[298]	validation_0-auc:0.978295	validation_1-auc:0.956394
[299]	validation_0-auc:0.978317	validation_1-auc:0.956373
[300]	validation_0

[425]	validation_0-auc:0.983726	validation_1-auc:0.960345
[426]	validation_0-auc:0.983763	validation_1-auc:0.960363
[427]	validation_0-auc:0.983837	validation_1-auc:0.960439
[428]	validation_0-auc:0.98385	validation_1-auc:0.960461
[429]	validation_0-auc:0.983891	validation_1-auc:0.960479
[430]	validation_0-auc:0.983931	validation_1-auc:0.960496
[431]	validation_0-auc:0.983956	validation_1-auc:0.960536
[432]	validation_0-auc:0.98397	validation_1-auc:0.96054
[433]	validation_0-auc:0.98398	validation_1-auc:0.960521
[434]	validation_0-auc:0.983991	validation_1-auc:0.960536
[435]	validation_0-auc:0.98414	validation_1-auc:0.960613
[436]	validation_0-auc:0.984161	validation_1-auc:0.96062
[437]	validation_0-auc:0.984244	validation_1-auc:0.960686
[438]	validation_0-auc:0.984297	validation_1-auc:0.960712
[439]	validation_0-auc:0.984333	validation_1-auc:0.96072
[440]	validation_0-auc:0.984348	validation_1-auc:0.960731
[441]	validation_0-auc:0.984368	validation_1-auc:0.960747
[442]	validation_0-au

[567]	validation_0-auc:0.988949	validation_1-auc:0.964093
[568]	validation_0-auc:0.98898	validation_1-auc:0.964131
[569]	validation_0-auc:0.988996	validation_1-auc:0.964124
[570]	validation_0-auc:0.988999	validation_1-auc:0.964126
[571]	validation_0-auc:0.989035	validation_1-auc:0.964155
[572]	validation_0-auc:0.98908	validation_1-auc:0.964222
[573]	validation_0-auc:0.989127	validation_1-auc:0.964269
[574]	validation_0-auc:0.989156	validation_1-auc:0.964284
[575]	validation_0-auc:0.989173	validation_1-auc:0.964302
[576]	validation_0-auc:0.989199	validation_1-auc:0.964301
[577]	validation_0-auc:0.989207	validation_1-auc:0.96432
[578]	validation_0-auc:0.989212	validation_1-auc:0.964323
[579]	validation_0-auc:0.989223	validation_1-auc:0.9643
[580]	validation_0-auc:0.98927	validation_1-auc:0.964303
[581]	validation_0-auc:0.989303	validation_1-auc:0.964332
[582]	validation_0-auc:0.98932	validation_1-auc:0.964352
[583]	validation_0-auc:0.989393	validation_1-auc:0.964413
[584]	validation_0-au

[709]	validation_0-auc:0.992066	validation_1-auc:0.966268
[710]	validation_0-auc:0.992071	validation_1-auc:0.966274
[711]	validation_0-auc:0.992079	validation_1-auc:0.96628
[712]	validation_0-auc:0.992085	validation_1-auc:0.96627
[713]	validation_0-auc:0.992129	validation_1-auc:0.966254
[714]	validation_0-auc:0.992155	validation_1-auc:0.966262
[715]	validation_0-auc:0.992158	validation_1-auc:0.966267
[716]	validation_0-auc:0.992168	validation_1-auc:0.966279
[717]	validation_0-auc:0.992251	validation_1-auc:0.966379
[718]	validation_0-auc:0.992261	validation_1-auc:0.96639
[719]	validation_0-auc:0.99228	validation_1-auc:0.96641
[720]	validation_0-auc:0.992303	validation_1-auc:0.96643
[721]	validation_0-auc:0.992324	validation_1-auc:0.966458
[722]	validation_0-auc:0.992362	validation_1-auc:0.966502
[723]	validation_0-auc:0.992389	validation_1-auc:0.966519
[724]	validation_0-auc:0.992403	validation_1-auc:0.966555
[725]	validation_0-auc:0.992408	validation_1-auc:0.966567
[726]	validation_0-a

[851]	validation_0-auc:0.993915	validation_1-auc:0.967696
[852]	validation_0-auc:0.993955	validation_1-auc:0.967747
[853]	validation_0-auc:0.993972	validation_1-auc:0.967742
[854]	validation_0-auc:0.993983	validation_1-auc:0.967761
[855]	validation_0-auc:0.993986	validation_1-auc:0.967764
[856]	validation_0-auc:0.993988	validation_1-auc:0.967762
[857]	validation_0-auc:0.994004	validation_1-auc:0.96777
[858]	validation_0-auc:0.994015	validation_1-auc:0.967774
[859]	validation_0-auc:0.994022	validation_1-auc:0.967787
[860]	validation_0-auc:0.994053	validation_1-auc:0.967813
[861]	validation_0-auc:0.994057	validation_1-auc:0.967812
[862]	validation_0-auc:0.994066	validation_1-auc:0.967801
[863]	validation_0-auc:0.994067	validation_1-auc:0.967806
[864]	validation_0-auc:0.994093	validation_1-auc:0.967822
[865]	validation_0-auc:0.994102	validation_1-auc:0.967813
[866]	validation_0-auc:0.994129	validation_1-auc:0.967815
[867]	validation_0-auc:0.994133	validation_1-auc:0.967821
[868]	validatio

[993]	validation_0-auc:0.995273	validation_1-auc:0.968531
[994]	validation_0-auc:0.995279	validation_1-auc:0.968538
[995]	validation_0-auc:0.995284	validation_1-auc:0.968543
[996]	validation_0-auc:0.995285	validation_1-auc:0.968541
[997]	validation_0-auc:0.995288	validation_1-auc:0.968544
[998]	validation_0-auc:0.99529	validation_1-auc:0.968548
[999]	validation_0-auc:0.995301	validation_1-auc:0.968571


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, eval_metric='auc',
              gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=2019,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, tree_method='gpu_exact', verbosity=1)

In [6]:
# Parameters for XGBoost - set tree_method='gpu_hist' to activate GPU usage
clf = xgb.XGBClassifier(
    n_estimators=10000,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=2019,
    tree_method='gpu_exact', #'gpu_exact', # THE MAGICAL PARAMETER
    eval_metric='auc', 
)

In [ ]:
# Time based split might be a better option, since it is closer to the real life test set
# eval_set = [(validation_set[features_columns], validation_set['isFraud'])]
# clf.fit(main_train_set[features_columns], main_train_set['isFraud'],early_stopping_rounds=100, eval_metric="auc", eval_set=eval_set, verbose=True)

# Looks like the random split without cross validation is too overfit. Went from ~0.97 to ~0.91 on the leaderboard
# eval_set = [(X_train,y_train),(X_test, y_test)]
# clf.fit(X_train, y_train,early_stopping_rounds=100, eval_metric="auc", eval_set=eval_set, verbose=True)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 20))
xgb.plot_importance(clf,max_num_features=50,ax=ax)

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/ieee-fraud-detection/sample_submission.csv', index_col='TransactionID')
sample_submission['isFraud'] = clf.predict_proba(test_full[features_columns])[:,1]
sample_submission.to_csv('simple_xgboost.csv')

In [ ]:
# sample_submission.head()